# Búsqueda en Anchura (Breadth-First Search, BFS) — Tutorial paso a paso

Este notebook presenta una implementación clara y didáctica del algoritmo de **Búsqueda en Anchura (BFS)**.

La BFS explora el espacio de estados **nivel por nivel**, garantizando encontrar el camino con **menor número de pasos** cuando todos los costos de las acciones son iguales.

## 0) Importaciones
Para BFS utilizaremos una **cola FIFO**. En Python, `collections.deque` es ideal para este propósito.


In [1]:
from collections import deque

## 1) Definición de la clase `Node`

Cada nodo de búsqueda contiene:
- `state`: el estado actual
- `parent`: referencia al nodo padre
- `action`: acción que llevó a este nodo

En BFS no es necesario almacenar el costo del camino.


In [2]:
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action


## 2) Función de expansión: `expand(problem, node)`

La expansión genera los nodos sucesores aplicando todas las acciones posibles desde el estado actual.


In [3]:
def expand(problem, node):
    s = node.state
    for action in problem.actions(s):
        s_prime = problem.result(s, action)
        yield Node(state=s_prime, parent=node, action=action)


## 3) Algoritmo de Búsqueda en Anchura (BFS)

Idea central:
- Utilizar una **cola FIFO** para expandir primero los nodos más antiguos.
- Explorar el espacio de estados **por niveles**.

Pasos:
1. Insertar el nodo inicial en la frontera.
2. Extraer el primer nodo de la cola.
3. Si es objetivo, retornar la solución.
4. Si no, expandirlo y añadir sus sucesores al final de la cola.


In [4]:
def breadth_first_search(problem):
    node = Node(problem.initial)

    if problem.is_goal(node.state):
        return node

    frontier = deque([node])
    reached = {problem.initial}

    while frontier:
        node = frontier.popleft()

        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                if problem.is_goal(s):
                    return child
                reached.add(s)
                frontier.append(child)

    return None


## 4) Definición de la clase `Problem`

Esta clase agrupa los elementos específicos del problema:
- estados inicial y objetivo
- acciones posibles
- función de transición
- prueba de objetivo


In [5]:
class Problem:
    def __init__(self, initial, goal, actions, result, is_goal):
        self.initial = initial
        self.goal = goal
        self.actions = actions
        self.result = result
        self.is_goal = is_goal


## 5) Ejemplo: mapa de Rumania

Usaremos el mismo mapa de Rumania, pero ahora BFS encontrará el camino con **menos ciudades intermedias**, no necesariamente el de menor distancia total.


In [6]:
ACTIONS = {
    "Arad": ["Sibiu", "Timisoara", "Zerind"],
    "Zerind": ["Arad", "Oradea"],
    "Oradea": ["Zerind", "Sibiu"],
    "Sibiu": ["Arad", "Oradea", "Fagaras", "Rimnicu Vilcea"],
    "Timisoara": ["Arad", "Lugoj"],
    "Lugoj": ["Timisoara", "Mehadia"],
    "Mehadia": ["Lugoj", "Drobeta"],
    "Drobeta": ["Mehadia", "Craiova"],
    "Craiova": ["Drobeta", "Rimnicu Vilcea", "Pitesti"],
    "Rimnicu Vilcea": ["Sibiu", "Craiova", "Pitesti"],
    "Fagaras": ["Sibiu", "Bucharest"],
    "Pitesti": ["Rimnicu Vilcea", "Craiova", "Bucharest"],
    "Bucharest": ["Fagaras", "Pitesti"]
}

def actions_fn(state):
    return ACTIONS.get(state, [])

def result_fn(state, action):
    return action

def is_goal_fn(state, goal="Bucharest"):
    return state == goal


## 6) Ejecutar BFS


In [7]:
initial = "Arad"
goal = "Bucharest"

problem = Problem(
    initial=initial,
    goal=goal,
    actions=actions_fn,
    result=result_fn,
    is_goal=lambda s: is_goal_fn(s, goal)
)

solution = breadth_first_search(problem)
solution


## 7) Reconstrucción del camino

BFS guarda punteros al padre, lo que permite reconstruir el camino solución.


In [8]:
def reconstruir_camino(goal_node):
    if goal_node is None:
        return None
    path = []
    node = goal_node
    while node is not None:
        path.append(node.state)
        node = node.parent
    path.reverse()
    return path

path = reconstruir_camino(solution)
path


['Arad', 'Sibiu', 'Fagaras', 'Bucharest']

## 8) Interpretación del resultado

- El camino obtenido tiene el **menor número de pasos**.
- No necesariamente es el de menor distancia total.
- BFS es óptimo solo cuando todos los costos son iguales.


In [9]:
if solution is None:
    print("No se encontró solución")
else:
    print("Camino solución (BFS):", path)
    print("Número de pasos:", len(path) - 1)


Camino solución (BFS): ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']
Número de pasos: 3


## 9) Comparación conceptual

- **BFS**: óptimo en número de pasos, alto consumo de memoria
- **DFS**: bajo consumo de memoria, no óptimo
- **Costo uniforme**: óptimo en costo
- **A\***: óptimo y eficiente con heurística admisible
